In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("pruned_offline_workload.CSV")
dev = pd.read_csv("online_workload_B.csv")
target = pd.read_csv("online_workload_C.csv")

In [3]:
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [4]:
#Initialisation and normalisation

workload_id = [df.workload_id[i] for i in range (18349)]
k1 = [df.k1[i] for i in range (18349)]
k2 = [df.k2[i] for i in range (18349)]
k3 = [df.k3[i] for i in range (18349)]
k4 = [df.k4[i] for i in range (18349)]
k5 = [df.k5[i] for i in range (18349)]
k6 = [df.k6[i] for i in range (18349)]
k7 = [df.k7[i] for i in range (18349)]
k8 = [df.k8[i] for i in range (18349)]
s1 = [df.s1[i] for i in range (18349)]
s2 = [df.s2[i] for i in range (18349)]
s3 = [df.s3[i] for i in range (18349)]
s4 = [df.s4[i] for i in range (18349)]
m1 = [df.m1[i]/1000000000 for i in range (18349)]
m2 = [df.m2[i]/100000000 for i in range (18349)]
m3 = [df.m3[i] * 100 for i in range (18349)]
latency = [df.latency[i] for i in range (18349)]
dev_m1 = [dev.m1[i]/1000000000 for i in range (600)]
dev_m2 = [dev.m2[i]/100000000 for i in range (600)]
dev_m3 = [dev.m3[i] * 100 for i in range (600)]
dev_latency = [dev.latency[i] for i in range (600)]
target_m1 = [target.m1[i]/1000000000 for i in range (500)]
target_m2 = [target.m2[i]/100000000 for i in range (500)]
target_m3= [target.m3[i] * 100 for i in range (500)]
target_latency = [target.latency[i] for i in range (500)]
target_workload =[target.workload_id for i in range(500)]
        

In [21]:
#Workload Mapping
from scipy.spatial import distance
mapped_workloads = []
for i in range(18349):
    min_avg=9999999
    index = -1
    for j in range(500):
        d1 = distance.euclidean(target_m1[j], m1[i])
        d2 = distance.euclidean(target_m2[j], m2[i])
        d3 = distance.euclidean(target_m3[j], m3[i])
        if (d1 + d2 + d3)/3 < min_avg:
            min_avg = (d1 + d2 + d3)/3
            index = j
    mapped_workloads.append(index)

        

In [5]:
X_train = [(m1[i], m2[i], m3[i]) for i in range (18349)]
X_test = [(target_m1[i], target_m2[i], target_m3[i]) for i in range (500)]
X_dev = [(dev_m1[i], dev_m2[i], dev_m3[i]) for i in range (600)]


In [ ]:
import pyGPs
import numpy as np
model = pyGPs.GPR()      # specify model (GP regression)
model.getPosterior(np.array(X_train), np.array(latency)) # fit default model (mean zero & rbf kernel) with data
model.optimize(np.array(X_train), np.array(latency))     # optimize hyperparamters (default optimizer: single run minimize)
model.predict(np.array(X_test))         # predict test cases
model.plot()             # and plot result

In [ ]:
import sklearn.gaussian_process as gp
kernel = gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0, (1e-3, 1e3))
model = gp.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1, normalize_y=True)
model.fit(X_train, latency)
params = model.kernel_.get_params()
y_pred, std = model.predict(X_test, return_std=True)

In [7]:
print(len(y_pred))

500


In [16]:
predicted_latency = []
for i in range (0, len(y_pred), 5):
    avg = (y_pred[i]+y_pred[i+1]+y_pred[i+2]+y_pred[i+3]+y_pred[i+4])/5
    predicted_latency.append(avg)
print(predicted_latency)


[9.40319083334, 36.401125, 22.799525, 37.9592550002, 34.2307775, 26.812803333600005, 12.935120000000001, 42.557264999999994, 24.290193333399998, 37.387489166659996, 34.839858333799995, 15.538705000000002, 34.15193000000001, 17.403772500080002, 8.739505, 386.63381, 8.856375, 343.37359499999997, 101.19203, 71.52809500000001, 54.83665499999999, 72.2965083334, 10.2497683334, 78.2180225004, 39.62437833399999, 47.130456666680004, 29.1843316664, 28.72006, 50.60391166666001, 30.5108483334, 25.813874999986666, 45.499275, 35.92258833340001, 25.508121666399997, 40.6385866666, 30.562540000000002, 62.08883500000002, 6.2338249999999995, 16.34196500048, 13.0548125, 39.599068333400005, 20.22753333334, 12.295225000000002, 152.68102833360004, 13.734585833259999, 324.7796, 110.788786667, 133.367605, 81.50443499999999, 135.549345, 9.9537841667, 78.55701, 23.92136, 61.35585000000001, 36.91566, 22.534625833339998, 12.186819999999999, 21.418305000600004, 14.689550000000002, 38.889775, 37.72936, 30.3102866668

In [18]:
import csv
rows = zip(predicted_latency)
with open('temp.CSV', mode='w') as csv_file:
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    for row in rows:
        wr.writerow(row)


In [26]:
import csv
work=[workload_id[mapped_workloads[i]] for i in range(100)]
rows = zip(work)
with open('temp1.CSV', mode='w') as csv_file:
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    for row in rows:
        wr.writerow(row)


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
RandomForest = RandomForestRegressor()
RandomForest.fit(X_train, latency)
y_pred = RandomForest.predict(X_test)

C:\Users\14135\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

np.random.seed(1)


def f(x):
    """The function to predict."""
    return x * np.sin(x)

# ----------------------------------------------------------------------
#  First the noiseless case
X = np.atleast_2d([1., 3., 5., 6., 7., 8.]).T

# Observations
y = f(X).ravel()

# Mesh the input space for evaluations of the real function, the prediction and
# its MSE
x = np.atleast_2d(np.linspace(0, 10, 1000)).T

# Instantiate a Gaussian Process model
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

# Fit to data using Maximum Likelihood Estimation of the parameters
gp.fit(X, y)

# Make the prediction on the meshed x-axis (ask for MSE as well)
y_pred, sigma = gp.predict(x, return_std=True)

# Plot the function, the prediction and the 95% confidence interval based on
# the MSE
plt.figure()
plt.plot(x, f(x), 'r:', label=r'$f(x) = x\,\sin(x)$')
plt.plot(X, y, 'r.', markersize=10, label='Observations')
plt.plot(x, y_pred, 'b-', label='Prediction')
plt.fill(np.concatenate([x, x[::-1]]),
         np.concatenate([y_pred - 1.9600 * sigma,
                        (y_pred + 1.9600 * sigma)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.ylim(-10, 20)
plt.legend(loc='upper left')

# ----------------------------------------------------------------------
# now the noisy case
X = np.linspace(0.1, 9.9, 20)
X = np.atleast_2d(X).T

# Observations and noise
y = f(X).ravel()
dy = 0.5 + 1.0 * np.random.random(y.shape)
noise = np.random.normal(0, dy)
y += noise

# Instantiate a Gaussian Process model
gp = GaussianProcessRegressor(kernel=kernel, alpha=dy ** 2,
                              n_restarts_optimizer=10)

# Fit to data using Maximum Likelihood Estimation of the parameters
gp.fit(X, y)

# Make the prediction on the meshed x-axis (ask for MSE as well)
y_pred, sigma = gp.predict(x, return_std=True)

# Plot the function, the prediction and the 95% confidence interval based on
# the MSE
plt.figure()
plt.plot(x, f(x), 'r:', label=r'$f(x) = x\,\sin(x)$')
plt.errorbar(X.ravel(), y, dy, fmt='r.', markersize=10, label='Observations')
plt.plot(x, y_pred, 'b-', label='Prediction')
plt.fill(np.concatenate([x, x[::-1]]),
         np.concatenate([y_pred - 1.9600 * sigma,
                        (y_pred + 1.9600 * sigma)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
plt.ylim(-10, 20)
plt.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

rbf = ConstantKernel(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=rbf, alpha=noise**2)

# Reuse training data from previous 1D example
gpr.fit(X_train, Y_train)

# Compute posterior predictive mean and covariance
mu_s, cov_s = gpr.predict(X, return_cov=True)

# Obtain optimized kernel parameters
l = gpr.kernel_.k2.get_params()['length_scale']
sigma_f = np.sqrt(gpr.kernel_.k1.get_params()['constant_value'])

# Compare with previous results
assert(np.isclose(l_opt, l))
assert(np.isclose(sigma_f_opt, sigma_f))

# Plot the results
plot_gp(mu_s, cov_s, X, X_train=X_train, Y_train=Y_train)